In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

### 1. Representing text as numerical data

In [7]:
# example text for model training (SMS messages)
simple_train = ['call you tonight', 'Call me a cab', 'Please call me... PLEASE!']

From the scikit-learn documentation:

Text Analysis is a major application field for machine learning algorithms. However the raw data, a sequence of symbols cannot be fed directly to the algorithms themselves as most of them expect numerical feature vectors with a fixed size rather than the raw text documents with variable length.

We will use CountVectorizer to "convert text into a matrix of token counts":

In [8]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [9]:
# learn the 'vocabulary' of the training data (occurs in-place)
vect.fit(simple_train)

CountVectorizer()

In [10]:
# examine the fitted vocabulary
vect.get_feature_names()

['cab', 'call', 'me', 'please', 'tonight', 'you']

In [11]:
# transform training data into a 'document-term matrix'
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [12]:
# convert sparse matrix to a dense matrix
simple_train_dtm.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]], dtype=int64)

In [13]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


From the scikit-learn documentation:

In this scheme, features and samples are defined as follows:

Each individual token occurrence frequency (normalized or not) is treated as a feature.
The vector of all the token frequencies for a given document is considered a multivariate sample.
A corpus of documents can thus be represented by a matrix with one row per document and one column per token (e.g. word) occurring in the corpus.

We call vectorization the general process of turning a collection of text documents into numerical feature vectors. This specific strategy (tokenization, counting and normalization) is called the Bag of Words or "Bag of n-grams" representation. Documents are described by word occurrences while completely ignoring the relative position information of the words in the document.

In [14]:
# check the type of the document-term matrix
type(simple_train_dtm)

scipy.sparse.csr.csr_matrix

In [15]:
# examine the sparse matrix contents
print(simple_train_dtm)

  (0, 1)	1
  (0, 4)	1
  (0, 5)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2


From the scikit-learn documentation:

As most documents will typically use a very small subset of the words used in the corpus, the resulting matrix will have many feature values that are zeros (typically more than 99% of them).

For instance, a collection of 10,000 short text documents (such as emails) will use a vocabulary with a size in the order of 100,000 unique words in total while each document will use 100 to 1000 unique words individually.

In order to be able to store such a matrix in memory but also to speed up operations, implementations will typically use a sparse representation such as the implementations available in the scipy.sparse package.

In [16]:
# example text for model testing
simple_test = ["please don't call me"]

In order to make a prediction, the new observation must have the same features as the training observations, both in number and meaning.

In [17]:
# transform testing data into a document-term matrix (using existing vocabulary)
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0, 0]], dtype=int64)

In [18]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


### Summary:

vect.fit(train) learns the vocabulary of the training data

vect.transform(train) uses the fitted vocabulary to build a document-term matrix from the training data

vect.transform(test) uses the fitted vocabulary to build a document-term matrix from the testing data (and ignores tokens it hasn't seen before)

In [21]:
# read file into pandas using a relative path
tweet = pd.read_csv("./train.csv")
# tweet.dropna(how="any", inplace=True, axis=1)
tweet.columns = ['id','keyword', 'location','text','target']
tweet.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


### Basic EDA

In [24]:
tweet.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [25]:
tweet.groupby('keyword').describe()

id                                              \
                    count          mean        std      min       25%   
keyword                                                                 
ablaze               36.0     70.388889  14.035216     48.0     58.50   
accident             35.0    121.800000  15.118746     96.0    109.50   
aftershock           34.0    171.323529  13.975564    146.0    160.25   
airplane%20accident  35.0    220.142857  15.406536    196.0    208.50   
ambulance            38.0    269.052632  14.101845    246.0    258.50   
...                   ...           ...        ...      ...       ...   
wounded              37.0  10609.135135  14.491688  10585.0  10598.00   
wounds               33.0  10662.393939  14.225724  10636.0  10651.00   
wreck                37.0  10708.513514  15.230856  10685.0  10695.00   
wreckage             39.0  10759.717949  14.730828  10735.0  10747.50   
wrecked              39.0  10810.692308  15.178159  10785.0  10798.50   

                                                target                      \
                         50%       75%      max  count      mean       std   
keyword                                                                      
ablaze                  69.5     81.25     95.0   36.0  0.361111  0.487136   
accident               121.0    134.50    145.0   35.0  0.685714  0.471008   
aftershock             171.5    182.75    195.0   34.0  0.000000  0.000000   
airplane%20accident    219.0    233.50    245.0   35.0  0.857143  0.355036   
ambulance              268.5    279.75    294.0   38.0  0.526316  0.506009   
...                      ...       ...      ...    ...       ...       ...   
wounded              10609.0  10622.00  10632.0   37.0  0.702703  0.463373   
wounds               10663.0  10675.00  10684.0   33.0  0.303030  0.466694   
wreck                10708.0  10722.00  10733.0   37.0  0.189189  0.397061   
wreckage             10760.0  10771.50  10784.0   39.0  1.000000  0.000000   
wrecked              10812.0  10823.50  10834.0   39.0  0.076923  0.269953   

                                              
                     min  25%  50%  75%  max  
keyword                                       
ablaze               0.0  0.0  0.0  1.0  1.0  
accident             0.0  0.0  1.0  1.0  1.0  
aftershock           0.0  0.0  0.0  0.0  0.0  
airplane%20accident  0.0  1.0  1.0  1.0  1.0  
ambulance            0.0  0.0  1.0  1.0  1.0  
...                  ...  ...  ...  ...  ...  
wounded              0.0  0.0  1.0  1.0  1.0  
wounds               0.0  0.0  0.0  1.0  1.0  
wreck                0.0  0.0  0.0  0.0  1.0  
wreckage             1.0  1.0  1.0  1.0  1.0  
wrecked              0.0  0.0  0.0  0.0  1.0  

[221 rows x 16 columns]